## **Regressão com PyCaret**

---
## **Importação das bibliotecas necessárias**

In [ ]:
!pip install pycaret
!pip install markupsafe==2.0.1

In [2]:
import pandas as pd
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## **Importação dos dados**

In [3]:
treino =  pd.read_csv('https://raw.githubusercontent.com/isabela-rossetti/FLAI-9-Competicao-ML/main/treino.csv')
teste =  pd.read_csv('https://raw.githubusercontent.com/isabela-rossetti/FLAI-9-Competicao-ML/main/teste.csv')

In [ ]:
treino

,hora,dia,feriado,estação,temperatura,chuva,umidade,sol,visibilidade,vento,aluguéis
0,16,sábado,não,verão,39.4,0.0,61.1,2.19,92.0%,3.49,1318
1,21,sexta,não,primavera,22.2,0.0,63.3,0.00,32.0%,2.89,686
2,16,segunda,não,verão,40.5,0.0,52.8,2.51,93.0%,3.97,831
3,7,segunda,sim,outono,25.6,0.0,70.9,0.00,97.0%,1.09,15
4,9,segunda,não,verão,39.9,0.0,58.1,1.69,62.0%,1.09,865
...,...,...,...,...,...,...,...,...,...,...,...
4495,6,sexta,não,verão,30.4,0.0,84.5,0.03,13.0%,1.57,723
4496,16,quarta,não,outono,28.5,0.0,48.3,0.87,67.0%,3.49,1151
4497,15,sexta,não,primavera,35.1,0.0,37.0,2.20,95.0%,3.49,1321
4498,18,domingo,não,outono,32.7,0.0,85.2,0.18,17.0%,3.85,533


In [ ]:
treino.describe()

,hora,temperatura,chuva,umidade,sol,vento,aluguéis
count,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000
mean,11.644889,27.108356,0.161864,62.087133,0.578104,2.322933,727.169556
std,6.901146,7.976684,1.179291,15.176118,0.878158,1.243427,646.886075
min,0.000000,6.500000,0.000000,18.100000,0.000000,0.250000,5.000000
25%,6.000000,20.900000,0.000000,50.500000,0.000000,1.330000,208.750000
50%,12.000000,27.800000,0.000000,61.100000,0.010000,2.050000,534.500000
75%,18.000000,33.500000,0.000000,73.900000,0.940000,3.010000,1096.000000
max,23.000000,44.600000,27.650000,92.000000,3.520000,9.130000,3566.000000


## **Alteração no tipo da variável 'visibilidade'**

In [4]:
treino ['visibilidade'] = treino ['visibilidade'].astype('str')
treino ['visibilidade'] = treino ['visibilidade'].str.replace('%', '')
treino ['visibilidade'] = pd.to_numeric(treino ['visibilidade'], errors='coerce')

In [5]:
teste ['visibilidade'] = teste ['visibilidade'].astype('str')
teste ['visibilidade'] = teste ['visibilidade'].str.replace('%', '')
teste ['visibilidade'] = pd.to_numeric(teste ['visibilidade'], errors='coerce')

## **Construção o modelo de previsão para demanda de bicicletas**

In [6]:
# configuração da regressão
regressao = setup(data = treino, target = 'aluguéis', normalize = True, session_id = 4703)

,Description,Value
0,session_id,4703
1,Target,aluguéis
2,Original Data,"(4500, 11)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3149, 19)"


In [7]:
# Comparando os modelos implementados, ordenando pelo RMSLE
melhores = compare_models(sort = 'RMSE', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,175.3618,85085.2249,290.8312,0.7953,0.8433,3.0064,0.085
et,Extra Trees Regressor,172.1843,87109.5668,294.2825,0.7907,0.8346,3.0455,0.811
rf,Random Forest Regressor,181.8522,93365.0778,304.9295,0.7757,0.8435,3.2066,1.010
gbr,Gradient Boosting Regressor,200.1588,103175.2326,320.5317,0.7522,0.8911,3.3961,0.220
knn,K Neighbors Regressor,269.9283,162994.7442,403.2209,0.6097,0.9390,3.0840,0.065
dt,Decision Tree Regressor,251.0007,185598.2419,429.5983,0.5558,1.1595,3.4518,0.021
lasso,Lasso Regression,345.3607,216292.5942,464.7611,0.4835,1.1115,3.8041,0.010
br,Bayesian Ridge,345.5106,216291.7673,464.7637,0.4835,1.1025,3.7988,0.011
lar,Least Angle Regression,345.8422,216360.5740,464.8330,0.4833,1.1039,3.8064,0.013
ridge,Ridge Regression,345.8268,216363.9863,464.8366,0.4833,1.1040,3.8061,0.009


In [8]:
lgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,183.2375,105487.6106,324.7886,0.7301,0.9490,4.6619
1,177.6147,103221.0881,321.2804,0.7368,0.8633,3.3833
2,166.7379,71617.3836,267.6142,0.8333,0.7318,1.8350
3,161.2156,71339.5323,267.0946,0.8416,0.8223,2.3970
4,169.1337,76984.8014,277.4614,0.8160,0.8904,4.2946
5,162.7786,70769.4394,266.0253,0.8462,0.7309,2.0039
6,182.1621,94067.0815,306.7036,0.7771,0.8753,3.6626
7,188.5817,84021.4633,289.8646,0.8009,0.8363,2.5779
8,185.7870,98256.9594,313.4597,0.7502,0.9789,3.5340


In [9]:
lgbm_tunado = tune_model(lgbm, optimize = 'RMSE', n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,182.3443,98859.3769,314.4191,0.7471,0.9109,4.0068
1,181.8857,102703.6871,320.4742,0.7381,0.8866,3.4478
2,172.1994,75674.9082,275.0907,0.8238,0.7526,1.8333
3,169.2522,82051.0143,286.4455,0.8178,0.8376,2.4392
4,164.0952,71361.9447,267.1366,0.8294,0.8634,3.9384
5,169.1111,74998.6453,273.8588,0.8370,0.7493,2.0654
6,195.9410,105951.3158,325.5016,0.7490,0.8355,3.5991
7,185.4855,84700.2794,291.0331,0.7993,0.8200,2.4628
8,182.5466,91336.4222,302.2192,0.7678,0.9800,3.3032


In [10]:
modelo_stack = stack_models(melhores)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,173.5874,97840.5688,312.7948,0.7497,0.9165,4.2998
1,171.3016,107205.9478,327.4232,0.7266,0.8343,3.6252
2,161.7379,72678.0515,269.5887,0.8308,0.7076,1.8123
3,151.7615,68698.9015,262.1048,0.8475,0.7906,2.3819
4,154.0024,66843.0220,258.5402,0.8402,0.8655,4.0532
5,153.4488,64447.8022,253.8657,0.8600,0.7236,1.9154
6,174.3507,88620.3202,297.6917,0.7900,0.8349,3.3255
7,179.5807,76900.5703,277.3095,0.8177,0.8274,2.7079
8,180.0078,92758.6373,304.5630,0.7642,0.9757,3.4197


## **Performance nos dados de validação**

In [11]:
pred = predict_model(modelo_stack)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,158.3994,73758.9547,271.586,0.8209,0.887,3.4024


In [12]:
pred2 = predict_model(lgbm)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,166.0891,78987.7825,281.0477,0.8082,0.9097,3.5848


In [13]:
pred3 = predict_model(lgbm_tunado)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,169.7358,80677.9956,284.0387,0.8041,0.9136,3.3522


## **Visualizando os dados de validação**

In [ ]:
#o modelo_stack apresentou melhor resultado na validação dos dados de teste
pred

,hora,temperatura,chuva,umidade,sol,visibilidade,vento,dia_domingo,dia_quarta,dia_quinta,...,dia_sexta,dia_sábado,dia_terça,feriado_sim,estação_inverno,estação_outono,estação_primavera,estação_verão,aluguéis,Label
0,1.648033,-0.292612,-0.131783,-0.102034,-0.666848,0.920428,-0.985300,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,911,923.358590
1,1.211818,0.574497,-0.131783,-0.598509,-0.610431,-0.030745,0.834696,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1907,1998.758980
2,-0.387639,-2.114798,-0.131783,-0.247667,-0.486313,0.580723,0.834696,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,241,284.820582
3,0.630197,1.944278,-0.131783,-0.896393,2.255562,0.920428,1.409432,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,866,755.728979
4,-0.242234,-0.732450,-0.131783,1.096125,-0.034975,0.920428,0.068382,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,247,281.517006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,1.211818,-1.310523,-0.131783,-1.101603,-0.666848,0.920428,-0.697933,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,274,356.436310
1347,-0.096829,0.675032,-0.131783,-0.102034,0.664598,0.920428,-0.218986,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1080,1053.062666
1348,1.066412,-0.166944,-0.131783,1.149082,-0.666848,0.308960,0.547328,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,297,449.150732
1349,-0.387639,0.926368,-0.131783,-1.048646,0.811282,0.818516,-0.602143,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1084,1284.921508


In [14]:
evaluate_model(modelo_stack)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## **Gerando e salvando o modelo final**

In [15]:
modelo_final = finalize_model(modelo_stack)

In [16]:
# Salvando o modelo
save_model(modelo_final, 'modelo')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='aluguéis',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strateg...
                                                                           tol=0.0001,
                                                                           validation_fraction=0.1,
                                                                           verbose=0,
                                        

---

## **Deploy Básico**

In [17]:
modelo = load_model('modelo')

pred = (predict_model(modelo, data = teste)['Label'].round())  

print('Aluguéis: {}'.format(pred)) 


Transformation Pipeline and Model Successfully Loaded
Aluguéis: 0        882.0
1        572.0
2        179.0
3        495.0
4       2177.0
         ...  
2995     373.0
2996     188.0
2997     257.0
2998     203.0
2999     151.0
Name: Label, Length: 3000, dtype: float64


In [18]:
previsoes = pd.DataFrame()
previsoes['aluguéis'] = pred
previsoes

,aluguéis
0,882.0
1,572.0
2,179.0
3,495.0
4,2177.0
...,...
2995,373.0
2996,188.0
2997,257.0
2998,203.0
